Load in the new hourly counts dataset:

In [35]:
import numpy as np
import pandas as pd

In [36]:
df = pd.read_csv('./data/hourlycounts_weather.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87631 entries, 0 to 87630
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                87631 non-null  int64  
 1   Unnamed: 0.1              87631 non-null  int64  
 2   year                      87631 non-null  int64  
 3   month                     87631 non-null  int64  
 4   day                       87631 non-null  int64  
 5   hour                      87631 non-null  int64  
 6   num_calls                 87631 non-null  int64  
 7   BRONX                     87631 non-null  int64  
 8   BROOKLYN                  87631 non-null  int64  
 9   MANHATTAN                 87631 non-null  int64  
 10  QUEENS                    87631 non-null  int64  
 11  RICHMOND / STATEN ISLAND  87631 non-null  int64  
 12  UNKNOWN                   87631 non-null  int64  
 13  STATION                   87631 non-null  object 
 14  NAME  

In [37]:
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn tools:
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
# sklearn models:
from sklearn.linear_model import LinearRegression
# tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input


Set up X and y, do train test split, and scale:

In [38]:
X = df[['year', 'month', 'day', 'hour', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TAVG_CALC']]
y = df['num_calls']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
sc = StandardScaler()
Z_train = sc.fit_transform(X_train)
Z_test = sc.transform(X_test)

Linear Modeling:

In [39]:
lr1 = LinearRegression()
lr1.fit(X_train,y_train)
R20 = lr1.score(X_train,y_train)
R21 = lr1.score(X_test,y_test)
RMS0= mean_squared_error(y_train,lr1.predict(X_train),squared=False)
RMS1= mean_squared_error(y_test, lr1.predict(X_test), squared=False)
print(f'R2:  Train: {R20}, Test: {R21}')
print(f'RMS: Train: {RMS0}, Test: {RMS1}')

R2:  Train: 0.4032671580422418, Test: 0.4052229110923856
RMS: Train: 40.255567545763256, Test: 40.11893624015415


We know several features have non-linear interactions with num_calls. Let's try Polynomial Features:

In [40]:
for n in range(2,5):
    print()
    print(f'Polynomial Features of Degree {n}:')
    pf = PolynomialFeatures(degree=n)
    PF_train = pf.fit_transform(Z_train)
    PF_test  = pf.transform(Z_test)
    lr2 = LinearRegression()
    lr2.fit(PF_train,y_train)
    R20 = lr2.score(PF_train,y_train)
    R21 = lr2.score(PF_test,y_test)
    RMS0= mean_squared_error(y_train,lr2.predict(PF_train),squared=False)
    RMS1= mean_squared_error(y_test, lr2.predict(PF_test), squared=False)
    print(f'R2:  Train: {R20}, Test: {R21}')
    print(f'RMS: Train: {RMS0}, Test: {RMS1}')


Polynomial Features of Degree 2:
R2:  Train: 0.5116451501308716, Test: 0.5154334226943383
RMS: Train: 36.416963952985625, Test: 36.211708980596406

Polynomial Features of Degree 3:
R2:  Train: 0.6730675026092716, Test: 0.6749179787498814
RMS: Train: 29.79648465778076, Test: 29.659837639626755

Polynomial Features of Degree 4:
R2:  Train: 0.7600247774298331, Test: 0.7575773505001402
RMS: Train: 25.52814072703261, Test: 25.612908048162943


Polynomial Features definitely helped!

Neural Net, first attempt:

In [41]:
#set random state for reproducability:
np.random.seed(42)

model = Sequential()
model.add(Dense(32, input_dim=Z_train.shape[1], 
                    activation='relu'))
model.add(Dense(16,  
                    activation='relu'))
model.add(Dense(8,  
                    activation='relu',))
model.add(Dense(1, activation=None))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])


results = model.fit(Z_train, y_train, epochs=100, batch_size=256, \
                            validation_data=(Z_test, y_test))



Epoch 1/100
257/257 [==============================] - 5s 12ms/step - loss: 25332.5375 - mae: 149.3183 - val_loss: 3636.4377 - val_mae: 48.2211
Epoch 2/100
257/257 [==============================] - 2s 6ms/step - loss: 2998.6525 - mae: 43.7047 - val_loss: 2260.6738 - val_mae: 38.1405
Epoch 3/100
257/257 [==============================] - 2s 7ms/step - loss: 2137.0729 - mae: 37.4904 - val_loss: 1978.6080 - val_mae: 36.0069
Epoch 4/100
257/257 [==============================] - 2s 6ms/step - loss: 1927.1506 - mae: 35.8471 - val_loss: 1853.7872 - val_mae: 35.0535
Epoch 5/100
257/257 [==============================] - 2s 7ms/step - loss: 1798.9916 - mae: 34.7096 - val_loss: 1791.2189 - val_mae: 34.5871
Epoch 6/100
257/257 [==============================] - 2s 6ms/step - loss: 1762.9768 - mae: 34.3062 - val_loss: 1745.7385 - val_mae: 34.20831763.1784 - mae: 34.305
Epoch 7/100
257/257 [==============================] - 1s 5ms/step - loss: 1730.6924 - mae: 34.0664 - val_loss: 1713.9470 - val_

In [42]:
preds_nn0 = model.predict(Z_train)
preds_nn1 = model.predict(Z_test)

In [43]:
R20 = lr2.score(PF_train,y_train)
R21 = lr2.score(PF_test,y_test)
RMS0= mean_squared_error(y_train,lr2.predict(PF_train),squared=False)
RMS1= mean_squared_error(y_test, lr2.predict(PF_test), squared=False)
print('Best Linear Model:')
print(f'R2:  Train: {R20}, Test: {R21}')
print(f'RMS: Train: {RMS0}, Test: {RMS1}')

Best Linear Model:
R2:  Train: 0.7600247774298331, Test: 0.7575773505001402
RMS: Train: 25.52814072703261, Test: 25.612908048162943


In [44]:
R20 = r2_score(y_train,preds_nn0)
R21 = r2_score(y_test, preds_nn1)
RMS0= mean_squared_error(y_train,preds_nn0,squared=False)
RMS1= mean_squared_error(y_test, preds_nn1, squared=False)
print('Neural Net Model:')
print(f'R2:  Train: {R20}, Test: {R21}')
print(f'RMS: Train: {RMS0}, Test: {RMS1}')

Neural Net Model:
R2:  Train: 0.7941486260633767, Test: 0.7953976735485776
RMS: Train: 23.64356000382835, Test: 23.530306132286523


So there is definitely some room for improvement, and we should be able to get a good model from this data.